In [1]:
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from IPython.display import display

In [2]:
def highlight_best(row):
    style = pd.Series(["normal"] * row.shape[0], index=row.index)
    if abs(row["Our Bias"]) < abs(row["CAE Bias"]):
        style["Our Bias"] = "bold"
    if row["Our Std"] < row["CAE Std"]:
        style["Our Std"] = "bold"
    return [f"font-weight: {weight}" for weight in style]

In [3]:
snr60_table = pd.DataFrame(
    {
        "CRB": [0.056, 0.78, 0.028, 0.054, 0.054],
        "CAE Bias": [-0.016, -0.48, -0.012, 0.001, -0.027],
        "CAE Std": [0.072, 0.67, 0.036, 0.053, 0.061],
    },
    index=["Flux [10^5]", "Sersic index", "Sersic radius [arcsec]", "g1", "g2"],
)
snr60_table.index.name = "SNR 60"

snr30_table = pd.DataFrame(
    {
        "CRB": [0.11, 1.56, 0.056, 0.11, 0.11],
        "CAE Bias": [-0.052, -0.57, 0.001, -0.019, 0.046],
        "CAE Std": [0.12, 0.85, 0.060, 0.10, 0.10],
    },
    index=["Flux [10^5]", "Sersic index", "Sersic radius [arcsec]", "g1", "g2"],
)
snr30_table.index.name = "SNR 30"

In [4]:
tests = [
    {"SNR": 60, "dataset": "../data/snr60.npz", "table": snr60_table},
    {"SNR": 30, "dataset": "../data/snr30.npz", "table": snr30_table},
]

modelpath = "xception_data_v2.tf"
scalerpath = "xception_data_v2.scaler"

In [6]:
for test in tests:
    # Load the data
    with np.load(test["dataset"]) as data:
        image = data["img"]
        label = data["label"]

    X = image[..., np.newaxis]
    # The labels are the same for every row in the test set. Take the first one.
    y_true = label[0]
    # Convert flux units to 10^5
    y_true[0] /= 1e5

    # Reload the scaler
    with open(scalerpath, "rb") as f:
        scaler = pickle.load(f)

    # Reload the model and make predictions
    model = tf.keras.models.load_model(modelpath)
    normalized_preds = model.predict(X)
    predictions = scaler.inverse_transform(normalized_preds)
    # Convert flux units to 10^5
    predictions[:, 0] /= 1e5

    # Show a benchmark table
    metrics = test["table"].copy()
    metrics["Our Bias"] = predictions.mean(axis=0) - y_true
    metrics["Our Std"] = predictions.std(axis=0)
    display(metrics.style.apply(highlight_best, axis=1).format("{:.3f}"))

,CRB,CAE Bias,CAE Std,Our Bias,Our Std
SNR 60,,,,,
Flux [10^5],0.056,-0.016,0.072,0.039,0.063
Sersic index,0.780,-0.480,0.670,0.435,0.765
Sersic radius [arcsec],0.028,-0.012,0.036,0.022,0.035
g1,0.054,0.001,0.053,0.011,0.048
g2,0.054,-0.027,0.061,-0.013,0.055


,CRB,CAE Bias,CAE Std,Our Bias,Our Std
SNR 30,,,,,
Flux [10^5],0.110,-0.052,0.120,0.052,0.098
Sersic index,1.560,-0.570,0.850,0.740,0.828
Sersic radius [arcsec],0.056,0.001,0.060,0.031,0.060
g1,0.110,-0.019,0.100,0.013,0.091
g2,0.110,0.046,0.100,-0.024,0.100
